In [1]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from ipywidgets import interact

shape = pd.read_csv('Shape.csv')

x_data_1 = np.array(shape['t_vortex_test'])
x_data_2 = np.array(shape['mPAP'])
shape_params = shape.iloc[:, 1:15]
y_data = np.array((shape_params)).T

selected_columns = shape.iloc[:, 1:15].columns

# Convert the column names to a list of strings
params = selected_columns.tolist()


In [3]:
from scipy.stats import linregress

r_squared_t =[]
r_squared_p =[]

for i in range(len(params)):

    slope, intercept, r_value_1, p_value, std_err = linregress(y_data[i], x_data_1)
    r2_1 = r_value_1**2
    slope, intercept, r_value_2, p_value, std_err = linregress(y_data[i], x_data_2)
    r2_2 = r_value_2**2
    
    r_squared_t.append(r2_1)
    r_squared_p.append(r2_2)
    
data = {'Parameters': params,
        'R_squared_T': r_squared_t,
       'R_squared_P': r_squared_p}

df = pd.DataFrame(data)


In [5]:
import plotly.graph_objects as go
from scipy.stats import linregress

def interactive_selection_plot(df, y_arrays):
  
  def update_plot(**kwargs):

    # Create the Figure
    fig = go.FigureWidget()
    filtered_df = pd.DataFrame()
    
    
    for i, (checkbox_name, selected) in enumerate(kwargs.items()):
        if i in range(len(df)):
          if selected:
            filtered_df = pd.concat([filtered_df, pd.DataFrame(df.loc[i]).T], ignore_index=True)
        
    # Create traces based on selected y-arrays
    traces = []
    selected_y_array = kwargs.get('y_array')
    for y_array in y_arrays:
        if selected_y_array == y_array:
            traces.append(go.Bar(x=filtered_df['Parameters'],
                                y=np.array(filtered_df[y_array]),
                                name=y_array))

    # Add traces to the figure
    fig.add_traces(traces)
    
    # Customize Layout
    fig.update_layout(
    title='Interactive Bar Chart with R-squared',
    xaxis_title='Parameter',
    yaxis_title='R2')
    
    # Display the Figure
    fig.show()

    
  # Create checkboxes for each data set
  checkboxes = {}
  for i, parameter in enumerate(df['Parameters']):
    checkbox = widgets.Checkbox(value=True, description=str(df['Parameters'][i]))
    checkboxes[f'checkbox_{i+1}'] = checkbox

  

  y_array_dropdown = widgets.Dropdown(
                options=y_arrays,
                description='Y-array:',
                value=y_arrays[0])  # Default selection
  checkboxes_tuple = tuple(checkboxes.values())
  layout = widgets.VBox([widgets.HBox(checkboxes_tuple), y_array_dropdown])
  interact(update_plot, **checkboxes, y_array=y_array_dropdown)


y_arrays = ['R_squared_T', 'R_squared_P']
interactive_selection_plot(df, y_arrays)





interactive(children=(Checkbox(value=True, description='Bif angle'), Checkbox(value=True, description='L_MPA')…